# Remote Sensing Hands-On Lesson, using TGO

 
   Planetary Data Workshop 4 Conference, Flagstaff, June 18, 2019
 
 
## Overview

 
   In this lesson you will develop a series of simple programs that
   demonstrate the usage of SpiceyPy to compute a variety of different
   geometric quantities applicable to experiments carried out by a remote
   sensing instrument flown on an interplanetary spacecraft. This
   particular lesson focuses on a spectrometer flying on the ExoMars2016 TGO
   spacecraft, but many of the concepts are easily extended and generalized
   to other scenarios.
   
   You may find it useful to consult the permuted index, the headers of
   various source modules, and several Required Reading documents available at 
   the NAIF site.

## Initialise SPICE by importing SpiceyPy

For the following exercises, instead of loading the meta-kernel try to sort out the exact kernels that you need to load for the given execrcise and load them (unless indicated).


In [1]:
import spiceypy 

## Time Conversion 


Write a program that given a UTC time string,
converts it to the following time systems and output formats:
 
* Ephemeris Time (ET) in seconds past J2000
* Calendar Ephemeris Time
* Spacecraft Clock Time
 
and displays the results. Use the program to convert "2018 JUN 11
19:32:00" UTC into these alternate systems.

In [2]:
#
# We need to load the leapseconds kernel and the SCLK kernel.
#
spiceypy.furnsh('kernels/lsk/naif0012.tls')
spiceypy.furnsh('kernels/sclk/em16_tgo_step_20160414.tsc')

et = spiceypy.utc2et('2018-06-11T19:32:00')
print('   Ephemeris Time (ET) in seconds past J2000: {}'.format(et))

calet = spiceypy.timout( et, 'YYYY-MON-DDTHR:MN:SC ::TDB' )
print( '   Calendar Ephemeris Time:                   {:s}'.format( calet ) )

#
# We will need the SCLK ID of TGO that we can retrieve from the SCLK file itself.
#
sclkid = -143

sclkst = spiceypy.sce2s( sclkid, et )
print( '   Spacecraft Clock Time:                     {:s}'.format( sclkst ) )

#
# We unload all the kernels in the kernel pool
#
spiceypy.kclear()

   Ephemeris Time (ET) in seconds past J2000: 582017589.1846417
   Calendar Ephemeris Time:                   2018-JUN-11T19:33:09
   Spacecraft Clock Time:                     1/0070841719.26698


## Obtaining Target States and Positions
 
Write a program that given a UTC time string computes the following quantities at that epoch:
 
* The apparent state of Mars as seen from ExoMars2016 TGO in the J2000 frame, in kilometers and kilometers/second. This vector itself is not of any particular interest, but it is a useful intermediate quantity in some geometry calculations.
  
* The one-way light time between ExoMars2016 TGO and the apparent position of Earth, in seconds.
  
* The actual (geometric) distance between the Sun and Mars, in astronomical units.
 
and displays the results. Use the program to compute these quantities at
"2018 JUN 11 19:32:00" UTC.

![title](img/observer_target.png)

In [3]:
#
# We need to load the leapseconds kernel the Solar System and Mars epehmeris and the TGO SPK.
#
spiceypy.furnsh('kernels/lsk/naif0012.tls')
spiceypy.furnsh('kernels/spk/de430.bsp')
spiceypy.furnsh('kernels/spk/mar085.bsp')
spiceypy.furnsh('kernels/spk/em16_tgo_mlt_20171205_20230115_v01.bsp')

et = spiceypy.utc2et('2018-06-11T19:32:00')

#
# Compute the apparent state of Mars as seen from ExoMars2016 TGO in the J2000 frame.  
# All of the ephemeris readers return states in units of kilometers and km/s.
#
[state, ltime] = spiceypy.spkezr('MARS', et,'J2000','LT+S','TGO' )
 
print( ' Apparent state of Mars as seen from ExoMars-16 TGO in the J2000\n'
       ' frame (km, km/s):')
print( '      X = {:16.3f}'.format(state[0])       )
print( '      Y = {:16.3f}'.format(state[1])       )
print( '      Z = {:16.3f}'.format(state[2])       )
print( '     VX = {:16.3f}'.format(state[3])       )
print( '     VY = {:16.3f}'.format(state[4])       )
print( '     VZ = {:16.3f}\n'.format(state[5])     )
 
#
# Compute the apparent position of Earth as seen from
# ExoMars2016 TGO in the J2000 frame.  
#
[pos, ltime] = spiceypy.spkpos( 'EARTH', et, 'J2000', 'LT+S', 'TGO')
 
print( ' One way light time between ExoMars2016 TGO and the apparent\n'
       ' position of Earth: {} seconds\n'.format(ltime))
 
#
# Now we need to compute the actual distance between the Sun and Mars.  
# We need to adjust our aberration correction appropriately.
#
[pos, ltime] = spiceypy.spkpos( 'SUN',  et, 'J2000','NONE', 'MARS')
 
#
# Compute the distance between the body centers in kilometers.
#
dist = spiceypy.vnorm( pos )
 
#
# Convert this value to AU using convrt.
#
dist = spiceypy.convrt( dist, 'KM', 'AU' )
 
print( ' Actual distance between Sun and Mars body centers: {} AU'.format(dist))
 
spiceypy.kclear()

 Apparent state of Mars as seen from ExoMars-16 TGO in the J2000
 frame (km, km/s):
      X =         2911.822
      Y =        -2033.802
      Z =        -1291.701
     VX =            1.310
     VY =           -0.056
     VZ =            3.104

 One way light time between ExoMars2016 TGO and the apparent
 position of Earth: 271.73803214866876 seconds

 Actual distance between Sun and Mars body centers: 1.4421944601935377 AU


## Spacecraft Orientation and Reference Frames

 
Write a program that given a UTC time string
computes and displays the following at the epoch of interest:
 
* The angular separation between the apparent position of Mars as seen from ExoMars2016 TGO and the nominal instrument view direction.
 
The nominal instrument view direction is not provided by any kernel variable, but it is indicated in the ExoMars2016 TGO frame kernel.
 
Use the program to compute these quantities at the epoch "2018 JUN 11 19:32:00" UTC.

![title](img/angular_separation.png)

In [4]:
#
# Since we need orientation infomration in addition to the kernels we loaded before
# now we need to load the Frames Kernel, the SCLK kernel and the TGO CK kernel.
#
spiceypy.furnsh('kernels/lsk/naif0012.tls')
spiceypy.furnsh('kernels/sclk/em16_tgo_step_20160414.tsc')
spiceypy.furnsh('kernels/spk/de430.bsp')
spiceypy.furnsh('kernels/spk/mar085.bsp')
spiceypy.furnsh('kernels/spk/em16_tgo_mlt_20171205_20230115_v01.bsp')
spiceypy.furnsh('kernels/fk/em16_tgo_v07.tf')
spiceypy.furnsh('kernels/ck/em16_tgo_sc_slt_npo_20171205_20230115_s20160414_v01.bc')
spiceypy.furnsh('kernels/pck/pck00010.tpc')

 
et = spiceypy.utc2et('2018-06-11T19:32:00')

#
# We compute the apparent position of Mars as seen from 
# ExoMars2016 TGO in the J2000 frame.
#
[pos, ltime] = spiceypy.spkpos('MARS',et,'J2000','LT+S','TGO')
 
#
# Now compute the location of the nominal instrument view
# direction.  From reading the frame kernel we know that
# the instrument view direction is nominally the -Y axis
# of the TGO_SPACECRAFT frame defined there.
#
bsight = [ 0.0, -1.0, 0.0]

#
# Now compute the rotation matrix from TGO_SPACECRAFT into
# J2000.
#
pform = spiceypy.pxform('TGO_SPACECRAFT','J2000', et )

#
# And multiply the result to obtain the nominal instrument
# view direction in the J2000 reference frame.
#
bsight = spiceypy.mxv(pform, bsight)

#
# Lastly compute the angular separation.
#
sep =  spiceypy.convrt(spiceypy.vsep(bsight, pos),'RADIANS','DEGREES')

print(' Angular separation between the apparent position of Mars and the\n'
      ' ExoMars2016 TGO nominal instrument view direction (degrees): {:.3f}'.format(sep))


spiceypy.kclear()

 Angular separation between the apparent position of Mars and the
 ExoMars2016 TGO nominal instrument view direction (degrees): 5.438


## Computing Sub-s/c and Sub-solar Points on an Ellipsoid and a DSK 

 
Write a program that given a UTC time string computes the following quantities at that epoch:
 
* The apparent sub-observer point of ExoMars2016 TGO on Mars, in the body fixed frame IAU_MARS, in kilometers.
* The apparent sub-solar point on Mars, as seen from ExoMars2016 TGO in the body fixed frame IAU_MARS, in kilometers.
 
The program computes each point twice: once using an ellipsoidal shape model and the
    
    near point/ellipsoid

definition, and once using a DSK shape model and the
    
    nadir/dsk/unprioritized

definition.

Use the provided meta-kernel to load the non-DSK kernels you need. Load the DSK kernels as needed.
The program displays the results. Use the program to compute these quantities at "2018 JUN 11 19:32:00" UTC.
   
![title](img/sub_solar_sc.png)

In [5]:
spiceypy.furnsh('remote_sensing_tgo.tm')

et = spiceypy.utc2et('2018-06-11T19:32:00')

for i in range(2):

    if i== 0:
        #
        # Use the "near point" sub-point definition
        # and an ellipsoidal model.
        #
        method = 'NEAR POINT/Ellipsoid'

    else:
        #
        # Use the "nadir" sub-point definition
        # and a DSK model.
        #
        method = 'NADIR/DSK/Unprioritized'
        spiceypy.furnsh('kernels/dsk/mars_lowres.bds')

    print( ' Sub-point/target shape model: {:s}\n'.format(method )  )

    #
    # Compute the apparent sub-observer point of ExoMars-16 TGO
    # on Mars.
    #
    [spoint, trgepc, srfvec] = spiceypy.subpnt(method,'MARS',et,
                                               'IAU_MARS','LT+S','TGO' )

    print( ' Apparent sub-observer point of ExoMars2016 TGO on Mars\n'
           ' in the IAU_MARS frame (km):' )
    print( '      X = {:.3f}'.format(spoint[0])              )
    print( '      Y = {:.3f}'.format(spoint[1])              )
    print( '      Z = {:.3f}'.format(spoint[2])              )
    print( '    ALT = {:.3f}\n'.format(spiceypy.vnorm(srfvec)) )

    #
    # Compute the apparent sub-solar point on Mars
    # as seen from ExoMars-16 TGO.
    #
    [spoint, trgepc, srfvec] = spiceypy.subslr(method,'MARS',et,'IAU_MARS',
                                               'LT+S','TGO' )

    print( ' Apparent sub-solar point on Mars as seen from ExoMars2016 \n'
           ' TGO in the IAU_MARS frame (km):'  )
    print( '      X = {:.3f}'.format(spoint[0])   )
    print( '      Y = {:.3f}'.format(spoint[1])   )
    print( '      Z = {:.3f}\n'.format(spoint[2])   )


spiceypy.unload('remote_sensing_tgo.tm')
spiceypy.unload('kernels/dsk/mars_lowres.tm')

 Sub-point/target shape model: NEAR POINT/Ellipsoid

 Apparent sub-observer point of ExoMars2016 TGO on Mars
 in the IAU_MARS frame (km):
      X = 2554.165
      Y = -2008.010
      Z = -983.240
    ALT = 385.045

 Apparent sub-solar point on Mars as seen from ExoMars2016 
 TGO in the IAU_MARS frame (km):
      X = 487.589
      Y = -3348.610
      Z = -286.697

 Sub-point/target shape model: NADIR/DSK/Unprioritized

 Apparent sub-observer point of ExoMars2016 TGO on Mars
 in the IAU_MARS frame (km):
      X = 2554.223
      Y = -2008.057
      Z = -983.263
    ALT = 384.967

 Apparent sub-solar point on Mars as seen from ExoMars2016 
 TGO in the IAU_MARS frame (km):
      X = 488.096
      Y = -3352.093
      Z = -286.999



## Intersecting Vectors with an Ellipsoid and a DSK (fovint)
 
   Write a program given input UTC time string computes the intersection of the ExoMars2016 TGO NOMAD LNO
   Nadir aperture boresight and field of view (FOV) boundary vectors with
   the surface of Mars with Mars' shape modeled by DSK data.
   The program presents each point of intersection as
 
* Planetocentric (latitudinal) coordinates in the IAU_MARS frame.
 
For each of the camera FOV boundary and boresight vectors, if an
   intersection is found, the program displays the results of the above
   computations, otherwise it indicates no intersection exists.
 
   At each point of intersection compute the following:
 
* Phase angle
* Solar incidence angle
* Emission angle

 
 Use this program to compute values at "2018 JUN 11 19:32:00" UTC.
 
![title](img/fov_intersection.png)

In [6]:
#
# We will have to handle errors
#
from spiceypy.utils.support_types import SpiceyError

spiceypy.furnsh('remote_sensing_tgo.tm')
spiceypy.furnsh('kernels/dsk/mars_lowres.bds')

et = spiceypy.utc2et('2018-06-11T19:32:00')

#
# Now we need to obtain the FOV configuration of
# the NOMAD LNO Nadir aperture.  To do this we will
# need the ID code for TGO_NOMAD_LNO_NAD.
#
lnonid = spiceypy.bodn2c('TGO_NOMAD_LNO_NAD')

#
# Now retrieve the field of view parameters.
#
[shape,insfrm,bsight,n,bounds ] = spiceypy.getfov(lnonid,4)

#
# `bounds' is a numpy array. We'll convert it to a list.
#
# Rather than treat BSIGHT as a separate vector,
# copy it into the last slot of BOUNDS.
#
bounds = bounds.tolist()
bounds.append( bsight )

#
# Set vector names to be used for output.
#
vecnam = [ 'Boundary Corner 1',
           'Boundary Corner 2',
           'Boundary Corner 3',
           'Boundary Corner 4',
           'TGO NOMAD LNO Nadir Boresight' ]

'DSK/Unprioritized'

#
# Get ID code of Mars. We'll use this ID code later, when we
# compute local solar time.
#
marsid = spiceypy.bodn2c( 'MARS' )

#
# Now perform the same set of calculations for each
# vector listed in the BOUNDS array. Use both
# ellipsoidal and detailed (DSK) shape models.
#
for i  in  range(5):
    #
    # Call sincpt to determine coordinates of the
    # intersection of this vector with the surface
    # of Mars.
    #
    print( ' Vector: {:s}\n'.format( vecnam[i] ) )

    try:

        [point,trgepc,srfvec] = spiceypy.sincpt('DSK/Unprioritized','MARS',et,
                                                   'IAU_MARS', 'LT+S','TGO',
                                                   insfrm, bounds[i])

        #
        # Display the planetocentric latitude and longitude of the intercept.
        #
        [radius,lon,lat] = spiceypy.reclat( point )

        print( ' Planetocentric coordinates of the intercept (degrees):')
        print( '     LAT = {:.3f}'.format(lat * spiceypy.dpr() )  )
        print( '     LON = {:.3f}\n'.format(lon * spiceypy.dpr() )  )
        
        #
        # Compute the illumination angles at this point.
        #
        [trgepc,srfvec,phase,solar,emissn, visibl,lit] = spiceypy.illumf(
                 'DSK/Unprioritized','MARS','SUN',et,'IAU_MARS', 
                 'LT+S','TGO', point)

        print( '  Phase angle (degrees): {:.3f}'.format(phase*spiceypy.dpr()))
        print( '  Solar incidence angle (degrees): {:.3f}'.format(solar*spiceypy.dpr()))
        print( '  Emission angle (degrees): {:.3f}'.format(emissn*spiceypy.dpr()))
        print( '  Observer visible:  {:s}'.format(str(visibl)))
        print( '  Sun visible:       {:s}\n'.format(str(lit)))


    except SpiceyError as exc:
        #
        # Display a message if an exception was thrown.
        # For simplicity, we treat this as an indication
        # that the point of intersection was not found,
        # although it could be due to other errors.
        # Otherwise, continue with the calculations.
        #
        print( 'Exception message is: {:s}'.format(exc.value ))
#
# End of vector loop.
#
              
spiceypy.kclear()

 Vector: Boundary Corner 1

 Planetocentric coordinates of the intercept (degrees):
     LAT = -16.967
     LON = -38.667

  Phase angle (degrees): 48.207
  Solar incidence angle (degrees): 44.387
  Emission angle (degrees): 4.326
  Observer visible:  True
  Sun visible:       True

 Vector: Boundary Corner 2

 Planetocentric coordinates of the intercept (degrees):
     LAT = -16.925
     LON = -38.962

  Phase angle (degrees): 50.707
  Solar incidence angle (degrees): 42.457
  Emission angle (degrees): 8.610
  Observer visible:  True
  Sun visible:       True

 Vector: Boundary Corner 3

 Planetocentric coordinates of the intercept (degrees):
     LAT = -16.917
     LON = -38.961

  Phase angle (degrees): 50.708
  Solar incidence angle (degrees): 42.457
  Emission angle (degrees): 8.593
  Observer visible:  True
  Sun visible:       True

 Vector: Boundary Corner 4

 Planetocentric coordinates of the intercept (degrees):
     LAT = -16.960
     LON = -38.665

  Phase angle (degrees): 